In [14]:
import pysam
import pandas as pd
from DetectSeqLib_V2.CheckAndLoadFiles import load_reference_fasta_as_dict
from DetectSeqLib_V2.RegionStatsTest import get_align_mismatch_pairs
from DetectSeqLib_V2.RegionStatsTest import get_No_MD_align_mismatch_pairs

In [15]:
vcf = "/Users/mac/mac_data/3.projects/25.2020-12_02_CBE_DetectSeq_bams/293T_BE_INPUT_VCF/293T-Mock-Input-covaris_bwa_hg38_sort_rmdup.recall.merge.Genotype.filter.rmdup_signal.vcf"
mut_bed = "/Users/mac/mac_data/3.projects/25.2020-12_02_CBE_DetectSeq_bams/293T_BE_INPUT_VCF/293T-EMX1-Mock-Input.site_index.rmdup.bed"

# ref_fasta_path = "/home/zhaohuanan/zhaohn_HD/2.database/fasta_hg38/hg38_only_chromosome.fa"

ref_fasta_path = "/Users/mac/Nutstore/Coding/github/snakepipes_bioinformatics_hermanzhaozzzz/genome_fa/genome_ucsc_hg38.fa"


with open(vcf, 'r') as vcf:
    ls_vcf = [i.strip().split('\t')[:2] + i.strip().split('\t')[3:5] for i in vcf.readlines() if i[0]!='#']
ls_filterd_vcf = [i for i in ls_vcf if (i[2]=='C' and i[3]=='T') or (i[2]=='G' and i[3]=='A')]

with open(mut_bed,'r') as bed:
    ls_bed = [i.strip().split('\t')[3].split('_') for i in bed.readlines()]
    ls_bed = [i[:2] + [j for j in i[2]] for i in ls_bed]

ls_endogenous_mut = ls_filterd_vcf + ls_bed
ls_endogenous_mut.sort()
ls_endogenous_mut[:10]

[['chr1', '1000018', 'G', 'A'],
 ['chr1', '10000789', 'G', 'A'],
 ['chr1', '10002030', 'G', 'A'],
 ['chr1', '100056713', 'G', 'A'],
 ['chr1', '100056713', 'G', 'A'],
 ['chr1', '100089610', 'C', 'T'],
 ['chr1', '100089610', 'C', 'T'],
 ['chr1', '100093453', 'G', 'A'],
 ['chr1', '100093453', 'G', 'A'],
 ['chr1', '100100720', 'C', 'T']]

In [16]:
dt_endogenous_mut = {}
for record in ls_endogenous_mut:
    dt_endogenous_mut["_".join(record[:3])+record[3]] = record
# dt_endogenous_mut

In [17]:
# 1
# path_bam = "./cbe_bam/293T-bat_EMX1-All-PD_rep1_hg38.MAPQ20.bam"
# bed_file = './bed/GSE151265_293T-EMX1-Detect-seq_pRBS.bed'
# 2
# path_bam = "./cbe_bam/293T-bat_EMX1-All-PD_rep2_hg38.MAPQ20.bam"
# bed_file = './bed/GSE151265_293T-EMX1-Detect-seq_pRBS.bed'
# 3
# path_bam = "./cbe_bam/293T-bat_HN-HEK4-All-PD_rep1_hg38.MAPQ20.bam"
# bed_file = './bed/GSE151265_293T-HEK4-Detect-seq_pRBS.bed'
# 4
# path_bam = "./cbe_bam/293T-bat_HN-HEK4-All-PD_rep2_hg38.MAPQ20.bam"
# bed_file = './bed/GSE151265_293T-HEK4-Detect-seq_pRBS.bed'
# 5
# path_bam = "./cbe_bam/MCF7-bat_HR-HEK4-All-PD_rep1_hg38.MAPQ20.bam"
# bed_file = './bed/GSE151265_MCF7-HEK4-Detect-seq_pRBS.bed'
# 6
# path_bam = "./cbe_bam/MCF7-bat_HR-HEK4-All-PD_rep2_hg38.MAPQ20.bam"
# bed_file = './bed/GSE151265_MCF7-HEK4-Detect-seq_pRBS.bed'
# 7
# path_bam = "./cbe_bam/MCF7-bat_HR-RNF2-All-PD_rep1_hg38.MAPQ20.bam"
# bed_file = './bed/GSE151265_MCF7-RNF2-Detect-seq_on-target.bed'
# 8
# path_bam = "./cbe_bam/MCF7-bat_HR-RNF2-All-PD_rep2_hg38.MAPQ20.bam"
# bed_file = './bed/GSE151265_MCF7-RNF2-Detect-seq_on-target.bed'


# test
path_bam = "/Users/mac/Nutstore/Coding/github/DetectSeq_mutation-region-count/bam/293T-bat_VEGFA-All-PD_rep1_hg38.MAPQ20.bam"
bed_file = "/Users/mac/Nutstore/Coding/github/DetectSeq_mutation-region-count/bed/GSE151265_293T-VEGFA-Detect-seq_pRBS.bed"

In [18]:
ref_dict = load_reference_fasta_as_dict(ref_fasta_path=ref_fasta_path)

INFO  @ 2020-12-03 21:13:28: Starting to load the reference genome... 
DEBUG @ 2020-12-03 21:13:30: Loading genome...	chr1 
DEBUG @ 2020-12-03 21:13:31: Loading genome...	chr2 
DEBUG @ 2020-12-03 21:13:32: Loading genome...	chr3 
DEBUG @ 2020-12-03 21:13:34: Loading genome...	chr4 
DEBUG @ 2020-12-03 21:13:35: Loading genome...	chr5 
DEBUG @ 2020-12-03 21:13:36: Loading genome...	chr6 
DEBUG @ 2020-12-03 21:13:37: Loading genome...	chr7 
DEBUG @ 2020-12-03 21:13:38: Loading genome...	chr8 
DEBUG @ 2020-12-03 21:13:39: Loading genome...	chr9 
DEBUG @ 2020-12-03 21:13:39: Loading genome...	chr10 
DEBUG @ 2020-12-03 21:13:40: Loading genome...	chr11 
DEBUG @ 2020-12-03 21:13:41: Loading genome...	chr12 
DEBUG @ 2020-12-03 21:13:42: Loading genome...	chr13 
DEBUG @ 2020-12-03 21:13:42: Loading genome...	chr14 
DEBUG @ 2020-12-03 21:13:43: Loading genome...	chr15 
DEBUG @ 2020-12-03 21:13:44: Loading genome...	chr16 
DEBUG @ 2020-12-03 21:13:44: Loading genome...	chr17 
DEBUG @ 2020-12-03 2

In [19]:
bam_file = pysam.AlignmentFile(path_bam, "rb")
num_extend = 100
has_bed_header = True
ls_mut_direction = ['CT','GA']

with open(bed_file, "r") as f:
    ls_bed_info = [i.strip().split('\t') for i in f.readlines()]

if has_bed_header:
    ls_bed_info = ls_bed_info[1:]
ls_bed_info[:3]

[['chr6', '43770819', '43770841', 'VEGFA-Site2-on-target'],
 ['chr1', '93448683', '93448705', 'VEGFA-Site2-pRBS-1'],
 ['chr15', '32993900', '32993922', 'VEGFA-Site2-pRBS-2']]

In [20]:
ls_chr = ['chr%s' % str(i+1) for i in range(22)] + ['chr%s' % i for i in ["X","Y","M"]]
ls_chr

['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr20',
 'chr21',
 'chr22',
 'chrX',
 'chrY',
 'chrM']

In [21]:
ls_table = []
for ls_bed_line in ls_bed_info:
    chr_ = ls_bed_line[0]
    start =int(ls_bed_line[1]) - num_extend
    end = int(ls_bed_line[2]) + num_extend
    region_index = ls_bed_line[3].strip()
    
    
    for index, align in enumerate(bam_file.fetch(contig=chr_, start=start, end=end)):
#         print(index, '\n', align, '\n', align.get_aligned_pairs())
        MD_tag_state = align.has_tag("MD")

        if MD_tag_state:
            mis_align_pair_list = get_align_mismatch_pairs(align)
        else:
            mis_align_pair_list = get_No_MD_align_mismatch_pairs(align, ref_dict)
#         print(mis_align_pair_list)
        # 0region_index, 1reads_id, 2align_strand, 3extend_length,4CT_count, 5Ct_max_gap, 6GA_count, 7GA_max_gap
        
        # 如果这个reads至少存在一个mutation信息
        if mis_align_pair_list and align.is_paired and not align.is_unmapped:
            # 获取chr
            chr_name = ls_chr[align.rname]
            # 去除mis align pair list中的固有mutations
            check_mis_align_pair_list = []
            for ls_mut in mis_align_pair_list:
                mut_key = "%s_%s_%s%s" % (chr_name, ls_mut[0], ls_mut[2], ls_mut[3])

                if not dt_endogenous_mut.__contains__(mut_key):
                    check_mis_align_pair_list.append(ls_mut)

            mis_align_pair_list = check_mis_align_pair_list

            # checked strand:
            if align.is_reverse:
                align_strand = "R"
            else:
                align_strand = 'F'

            aligned_reads_id = align.query_name
            extend_length = num_extend

            # 如果只存在一个mutation，不做判断，直接存入list
            if len(mis_align_pair_list)==1:
                ls_line = []
                if mis_align_pair_list[0][2]=='C' and mis_align_pair_list[0][3]=='T':
                    ls_line = [region_index, aligned_reads_id, align_strand, extend_length, 1, 0, 0, 0]
                elif mis_align_pair_list[0][2]=='G' and mis_align_pair_list[0][3]=='A':
                    ls_line = [region_index, aligned_reads_id, align_strand, extend_length, 0, 0, 1, 0]

            # 如果存在两个或两个以上的mutation        
            else:
                ls_line1 = []
                ls_line2 = []
                ls_line = []
                # 先把CT的全筛出来
                ls_this_reads_info = [site for site in mis_align_pair_list if site[2]=='C' and site[3]=='T']
                # 如果筛不到，没有就没有
                # 如果筛到至少一个
                if ls_this_reads_info:
                    # 如果有一个就加到ls_table中
                    if len(ls_this_reads_info)==1:
                        ls_line = [region_index, aligned_reads_id, align_strand, extend_length, 1, 0, 0, 0]
                    # 如果有更多再计算count和gap
                    else:# todo 
                        ls_first = ls_this_reads_info[0]
                        ls_last = ls_this_reads_info[-1]
#                         print(ls_first[1])# todo!
                        ls_line1 = [region_index, aligned_reads_id, align_strand, extend_length, len(ls_this_reads_info),ls_last[0]-ls_first[0],0,0]
                # 再把GA的全筛出来
                ls_this_reads_info = [site for site in mis_align_pair_list if site[2]=='G' and site[3]=='A']
                # 如果筛不到，没有就没有
                # 如果筛到至少一个
                if ls_this_reads_info:
                    # 如果有一个就加到ls_table中
                    if len(ls_this_reads_info)==1:
                        ls_line = [region_index, aligned_reads_id, align_strand, extend_length, 0, 0, 1, 0]
                    # 如果有更多再计算count和gap
                    else:# todo 
                        ls_first = ls_this_reads_info[0]
                        ls_last = ls_this_reads_info[-1]
#                         print(ls_first[1])# todo!
                        ls_line2 = [region_index, aligned_reads_id, align_strand, extend_length,0,0,len(ls_this_reads_info),ls_last[0]-ls_first[0]]
                if ls_line1 and ls_line2:
                    ls_line = [region_index, aligned_reads_id, align_strand, extend_length,ls_line1[4]+ls_line2[4], ls_line1[5]+ls_line2[5],ls_line1[6]+ls_line2[6], ls_line1[7]+ls_line2[7]]
                elif ls_line1:
                    ls_line = ls_line1
                elif ls_line2:
                    ls_line = ls_line2
                else:
                    pass
                if ls_line:
                    ls_table.append(ls_line)
                    
                # 既没有CT也没有GA的直接pass

In [22]:
df = pd.DataFrame(ls_table, columns=["region_index", "reads_id", "align_orientation", "extend_length", "count_C2T", "max_distance_C2T", "count_G2A", "max_distance_G2A"])
df

,region_index,reads_id,align_orientation,extend_length,count_C2T,max_distance_C2T,count_G2A,max_distance_G2A
0,VEGFA-Site2-on-target,E00454:613:H527LCCX2:6:1122:17756:71946,F,100,1,0,0,0
1,VEGFA-Site2-on-target,E00454:613:H527LCCX2:6:2222:24302:20348_1:N:0:...,F,100,0,0,1,0
2,VEGFA-Site2-on-target,E00454:613:H527LCCX2:6:2106:4137:42306_1:N:0:A...,F,100,1,0,0,0
3,VEGFA-Site2-on-target,E00454:613:H527LCCX2:6:1107:25936:21350_1:N:0:...,F,100,0,0,2,1
4,VEGFA-Site2-on-target,E00454:613:H527LCCX2:6:1106:2879:61608_1:N:0:A...,F,100,0,0,3,4
...,...,...,...,...,...,...,...,...
53211,VEGFA-Site2-pRBS-347,E00454:613:H527LCCX2:6:1124:23723:22405_1:N:0:...,R,100,0,0,2,7
53212,VEGFA-Site2-pRBS-347,E00490:581:H52CJCCX2:4:2116:16386:66302_1:N:0:...,R,100,2,4,0,0
53213,VEGFA-Site2-pRBS-347,E00490:581:H52CJCCX2:4:1218:22181:49197_1:N:0:...,R,100,0,0,5,7
53214,VEGFA-Site2-pRBS-347,E00490:581:H52CJCCX2:4:2213:14215:55473_1:N:0:...,R,100,0,0,2,2


In [23]:
csv_path = path_bam.replace("cbe_bam", "table").replace("_hg38.MAPQ20.bam", "_max_distance_count.csv")

In [24]:
df.to_csv(csv_path, sep="\t")

In [25]:
bam_file.close()
